In [ ]:
# ----------------------------------------------------------------------
# 필수 라이브러리 설치 및 불러오기
# ----------------------------------------------------------------------
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth       # FP-Growth 함수
from mlxtend.frequent_patterns import association_rules # 연관 규칙 생성 함수

# ----------------------------------------------------------------------
# 데이터 준비 (장바구니 거래 데이터)
# ----------------------------------------------------------------------
transactions = [
    ['맥주', '기저귀', '과자'],
    ['맥주', '기저귀', '사이다'],
    ['맥주', '우유'],
    ['기저귀', '우유', '과자'],
    ['맥주', '기저귀', '과자', '우유']
]

# ----------------------------------------------------------------------
# 1단계: 데이터 전처리 (One-Hot Encoding)
# ----------------------------------------------------------------------
# FP-Growth는 항목의 존재 여부를 나타내는 True/False 매트릭스를 입력으로 받음.
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_one_hot = pd.DataFrame(te_ary, columns=te.columns_)

print("--- 1단계: One-Hot Encoding 데이터 ---")
print(df_one_hot)

# ----------------------------------------------------------------------
# 2단계: FP-Growth로 빈발 항목 집합 찾기
# ----------------------------------------------------------------------
# fpgrowth 함수를 사용하여 최소 지지도(min_support) 기준 만족 항목 집합 도출.
# 이 과정에서 내부적으로 FP-Tree 구조가 생성되어 탐색됨.
MIN_SUPPORT = 0.6 # 최소 지지도 60% 설정
frequent_itemsets_fp = fpgrowth(df_one_hot, 
                                min_support=MIN_SUPPORT, 
                                use_colnames=True)

print("\n--- 2단계: 빈발 항목 집합 (Frequent Itemsets) ---")
print(frequent_itemsets_fp.sort_values(by='support', ascending=False))

# ----------------------------------------------------------------------
# 3단계: 연관 규칙 (Association Rules) 생성 및 정리
# ----------------------------------------------------------------------
# 최소 신뢰도(min_threshold)를 기준으로 연관 규칙 생성.
MIN_CONFIDENCE = 0.75 # 최소 신뢰도 75% 설정
rules_fp = association_rules(frequent_itemsets_fp, 
                            metric="confidence", 
                            min_threshold=MIN_CONFIDENCE)

# 결과를 핵심 지표(antecedents, consequents, support, confidence, lift)만 선택하여 정리
rules_fp_final = rules_fp[[
    'antecedents',     # 선행 항목 (IF)
    'consequents',     # 후행 항목 (THEN)
    'support',         # 지지도
    'confidence',      # 신뢰도 (규칙의 정확도)
    'lift'             # 향상도 (연관성의 강력함)
]].sort_values(by='confidence', ascending=False)

print("\n--- 3단계: 최종 연관 규칙 (핵심 지표) ---")
print(rules_fp_final)